In [29]:
# Install the docx2txt package
!pip install docx2txt
# Import the library
import docx2txt


  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3966 sha256=fd50cf4560fe3321156dce548d28eda55b245f40626c5ab8a44192fe6bcf6d81
  Stored in directory: /Users/alyssiaoh/Library/Caches/pip/wheels/40/75/01/e6c444034338bde9c7947d3467807f889123465c2371e77418
Successfully built docx2txt


In [33]:

from sklearn.linear_model import LinearRegression, LogisticRegression
from scipy import stats

from sklearn.metrics import (accuracy_score, 
                            confusion_matrix, 
                            classification_report,
                            plot_confusion_matrix)
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns


import requests
from bs4 import BeautifulSoup    
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import FreqDist, pos_tag
from collections import Counter
import re
from sklearn.metrics import mean_squared_error 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
# Import CountVectorizer and TFIDFVectorizer from feature_extraction.text.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)


%config InlineBackend.figure_format = 'retina'

In [22]:
df = pd.read_csv('jobs_summarized.csv', )
df = df.drop(columns=['Unnamed: 0'])
df[20:25]

title  \
20  Data Analyst (Entry) - Data Science - R&D - PH - US_new   
21                                      Payroll Manager_new   
22   Program Manager, Business Operations, Data Centers_new   
23                              Contracts Administrator_new   
24                                Development Assistant_new   

                                                   company  \
20                                                   Bayer   
21  Golden Gate Bridge Highway and Transportation District   
22                                                  Google   
23                       Banking and Finance Professionals   
24                                                  On Lok   

                                             location  \
20                                 Berkeley, CA 94710   
21                  San Rafael, CA 94901 (Canal area)   
22                                  San Francisco, CA   
23                              Foster City, CA 94404   
24  San Francisco, CA 94111 (Financial District area)   

                                                                                                                                                            summary  \
20             Experience with data dashboarding (Spotfire, tableau, or similar).\nWork with users (scientists) to ensure the tools we build are usable and useful.   
21   Maintain control of payroll system; conduct regular system and process audits to ensure data integrity and compliance.\nPosition is located in San Rafael, CA.   
22                                  Bachelor's degree in Technology, Engineering, Science, Economics, Business or related field or equivalent practical experience.   
23  Ø Conduct analysis and maintain database systems related to contracts and proposals.\nØ Perform data integrity and quality control checks to identify areas of…   
24           Proven track record of meeting goals doing data entry and administrative duties as a Data Entry operator, Office Administrator or equivalent position.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [25]:
df['description'].dropna(inplace=True)

In [27]:
df_norating = df[df['rating'] == 'None']
df_rating = df[df['rating'] != 'None']

In [32]:
df_rating.to_csv('jobs_df_rating.csv', index=False)

In [31]:
df_norating.to_csv('jobs_df_norating.csv', index=False)

In [33]:
df_rating = pd.read_csv('jobs_df_rating.csv')

In [43]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559 entries, 0 to 558
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            559 non-null    object 
 1   company          559 non-null    object 
 2   location         559 non-null    object 
 3   summary          559 non-null    object 
 4   description      544 non-null    object 
 5   salary           559 non-null    object 
 6   rating           559 non-null    float64
 7   desc_summarizer  559 non-null    object 
 8   rating_class     559 non-null    object 
dtypes: float64(1), object(8)
memory usage: 39.4+ KB


In [40]:
df_rating['rating'] = df_rating['rating'].astype(float)

In [4]:
def classify_rating(num):
    if float(num) < 3.5:
        return 'below average'
    elif float(num) > 4:
        return 'above average'
    else:
        return 'average'


df_rating['rating_class'] = df_rating['rating'].apply(lambda x: classify_rating(x))

In [3]:
df_rating = pd.read_csv('data_glassdoor.csv')

In [152]:
df_rating.head()

,salary_min,salary_max,description,rating,company,title,salary_avg,rating_class
0,37000.0,66000.0,"Are you eager to roll up your sleeves and harness data to drive policy change? Do you enjoy sifting through complex datasets to illuminate trends and insights? Do you see yourself working for a values-driven organization with a vision to tackle the most pressing injustices of our day?\n\nWe are looking to hire a bright, hard-working, and creative individual with strong data management skills a...",3.2,Vera Institute of Justice,"Data Analyst, Center on Immigration and Justice (CIJ)",51500.0,below average
1,37000.0,66000.0,"Overview\n\nProvides analytical and technical support for the integration of multiple data sources used to prepare internal and external reporting for the Quality Management team and business stakeholders. Provides support and analytical insight for Quality Incentive measures, HEDIS measures, and Quality Improvement initiatives. Monitors, analyzes, and communicates Quality performance related ...",3.8,Visiting Nurse Service of New York,Quality Data Analyst,51500.0,average
2,37000.0,66000.0,"We’re looking for a Senior Data Analyst who has a love of mentorship, data visualization, and generating actionable insights from raw data. In this role, you’ll have the opportunity to be an organizational influencer, who will generate insights with a good degree of autonomy, and partner with data science to grow deeper analytical skills. You will be joining the Insights & Analytics team, a te...",3.4,Squarespace,"Senior Data Analyst, Insights & Analytics Team [Customer Operations]",51500.0,below average
3,37000.0,66000.0,"Requisition NumberRR-0001939\nRemote:Yes\nWe collaborate. We create. We innovate.\n\nIntrigued?\n\nYou’re a business professional with an innate curiosity that thrives in a dynamic and Agile environment. You appreciate teamwork, exemplify integrity, perseverance, flexibility, and a generosity of spirit… if this sounds like you, then please apply – we’d love to meet you!\n\nCelerity is expandin...",4.1,Celerity,Data Analyst,51500.0,above average
4,37000.0,66000.0,"ABOUT FANDUEL GROUP\n\nFanDuel Group is a world-class team of brands and products all built with one goal in mind — to give fans new and innovative ways to interact with their favorite games, sports, teams, and leagues. That's no easy task, which is why we're so dedicated to building a winning team. And make no mistake, we are here to win, but we believe in winning right. That means we'll neve...",3.9,FanDuel,Reporting Data Analyst,51500.0,average


In [121]:
df_rating.columns

Index(['title', 'company', 'location', 'summary', 'description', 'salary',
       'rating', 'desc_summarizer', 'rating_class'],
      dtype='object')

In [5]:
df_rating['rating_class'].value_counts()


average          5849
below average    5505
above average    4093
Name: rating_class, dtype: int64

In [7]:
X = df_rating[['company']]
X = pd.get_dummies(data=X, drop_first=True, columns=['company'])
y = df_rating['rating_class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
X_train.shape

(12357, 5959)

In [151]:
y_train.shape

(12357,)

In [9]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:

knn3 = KNeighborsClassifier(n_neighbors=3)
knn3.fit(X_train, y_train)


knn5 = KNeighborsClassifier(n_neighbors=5)
knn5.fit(X_train, y_train)


knn15 = KNeighborsClassifier(n_neighbors=15)
knn15.fit(X_train, y_train)


knn25 = KNeighborsClassifier(n_neighbors=25)
knn25.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=25)

In [11]:
logreg_ridge1 = LogisticRegression()
logreg_ridge1.fit(X_train, y_train)

logreg_ridge10 = LogisticRegression(C = .1)
logreg_ridge10.fit(X_train, y_train)

logreg_lasso1 = LogisticRegression(penalty = 'l1', C = 1, solver='liblinear')
logreg_lasso1.fit(X_train, y_train)

logreg_lasso10 = LogisticRegression(penalty = 'l1', C = .1, solver='liblinear')
logreg_lasso10.fit(X_train, y_train)

/Users/alyssiaoh/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.1, penalty='l1', solver='liblinear')

In [12]:
def model_cv(model, X_train, X_test, y_train, y_test):
    cv_train = cross_val_score(model, X_train, y_train).mean()
    cv_test = cross_val_score(model, X_test, y_test).mean()
    return print(f'Model: {model}, mean CV score of train set: {cv_train}, mean CV score of test set: {cv_test}, difference test-train: {cv_test - cv_train}')
    
model_cv(knn3, X_train, X_test, y_train, y_test)
model_cv(knn5, X_train, X_test, y_train, y_test)
model_cv(knn15, X_train, X_test, y_train, y_test)
model_cv(knn25, X_train, X_test, y_train, y_test)
model_cv(logreg_ridge1, X_train, X_test, y_train, y_test)
model_cv(logreg_ridge10, X_train, X_test, y_train, y_test)
model_cv(logreg_lasso1, X_train, X_test, y_train, y_test)
model_cv(logreg_lasso10, X_train, X_test, y_train, y_test)


Model: KNeighborsClassifier(n_neighbors=3), mean CV score of train set: 0.7327839507870587, mean CV score of test set: 0.5223300970873785, difference test-train: -0.21045385369968017
Model: KNeighborsClassifier(), mean CV score of train set: 0.6677205748494838, mean CV score of test set: 0.5042071197411003, difference test-train: -0.16351345510838344
Model: KNeighborsClassifier(n_neighbors=15), mean CV score of train set: 0.44557861484482125, mean CV score of test set: 0.5207119741100323, difference test-train: 0.07513335926521109
Model: KNeighborsClassifier(n_neighbors=25), mean CV score of train set: 0.36481404355245767, mean CV score of test set: 0.4932038834951456, difference test-train: 0.12838983994268793


/Users/alyssiaoh/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/alyssiaoh/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

Model: LogisticRegression(), mean CV score of train set: 0.8405774623169215, mean CV score of test set: 0.6977346278317151, difference test-train: -0.1428428344852064


/Users/alyssiaoh/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/alyssiaoh/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

Model: LogisticRegression(C=0.1), mean CV score of train set: 0.8175142658069857, mean CV score of test set: 0.6773462783171521, difference test-train: -0.1401679874898336
Model: LogisticRegression(C=1, penalty='l1', solver='liblinear'), mean CV score of train set: 0.8404965234257844, mean CV score of test set: 0.6796116504854368, difference test-train: -0.1608848729403476
Model: LogisticRegression(C=0.1, penalty='l1', solver='liblinear'), mean CV score of train set: 0.8405774295746518, mean CV score of test set: 0.6773462783171521, difference test-train: -0.16323115125749965


In [13]:
print(f'Baseline accuracy of default model is {5849/12357}')

Baseline accuracy of default model is 0.4733349518491543


In [30]:
# Store the resume in a variable
resume = docx2txt.process("alyssiaoh_resume.docx")

# Print the resume
print(resume)

Alyssia Oh

San Francisco, CA 8312393523

alyssiao@gmail.com • linkedin.com/in/alyssiao • Github/portfolio



I am a data scientist with a passion for translating my scientific mind into actionable insights. I come from a background in biomedical research and drug development where I’ve been trained on how to think about data as well as how to design experiments to obtain the desired data, and what the next step should be to strengthen the solution. To facilitate communicating complex ideas, I like using data visualization tools to provide an intuitive understanding of the topic to all levels of expertise.  





Skills



Languages/Environments | Python, SQL, R, Google Colab

Packages/Programs | Pandas, NumPy, Scikit-learn (sklearn), Scipy, Biopython, Requests, NLTK, BeautifulSoup, PySpark, TensorFlow, Keras, BeautifulSoup, AWS, Heroku, Flask, Gensim, pymc3, GraphPad Prism, Pharsight Phoenix WinNonlin, FlowJo, DNAStar

Data Visualization | Seaborn, Plotly, Tableau, matplotlib, ggplot1

In [36]:
# Store the job description into a variable
job_description = '''
The Clinical Trial Research Coordinator I (CTRC-I) is a novice level position which, under the clinical direction of the Principal Investigator (PI), assists in the operational and administrative research support of the Kaiser Permanente Northern California (KPNC) PI and KPNC Clinical Trials Operations Director or designee, and other research staff conducting multiple FDA-regulated clinical trials. This is a non-licensed position which works under the daily supervision of an experienced Clinical Trial Nurse, Clinical Trial Research Coordinator, and/or KFRI-designated, licensed clinical trial research staff member. This position must adhere to the position's scope of practice as outlined in the Major Responsibilities below. This position is also responsible for assisting in protecting the health, safety, and welfare of research participants.\n\n\n\nEssential Functions:\nComplianceSupport & comply w/ the Principles of Responsibility (Kaiser Permanente Code of Conduct). W/ guidance from PI, assist w/ ensuring compliance w/ KPNC IRB Standard Operating Procedures (SOP) & document applications. Adhere to Guideline for Good Clinical Practice (GCP), federal, state, & local regulations, & KP policies & procedures. W/ supervision, assist w/ preparation for inspections, audits & monitoring visits.Study Implementation: Maintain the security & confidentiality of participants' paper or electronic data (e.g., case report forms kept in a secure, locked space). Assist w/ scheduling participants for study assessments/visits, required tests including visit-specific laboratory kits and/or paperwork preparation. Prepare packets for participants (e.g., instructions, phone numbers, calendars, diaries, etc.).\nAssist w/ obtaining medical records & test results for all projects/participants. w/ supervision & certification, perform packaging & shipping of protocol specimens to the Sponsor lab in accordance w/ IATA/DOT regulations & Sponsor shipping guidelines. Direct issues requiring medical decision-making to the appropriate licensed staff member promptly.\nProvide PI and/or Clinical Trial Nurse w/ the ongoing informed consent process by coordinating the current consent documents. According to protocol and/or IRB-approved telephone script, collect & document research data in a timely manner, & report the information to the appropriate licensed staff member & PI for assessment. Assist in the collection of protocol-required data w/ timely & accurate submissions as required by Sponsor (e.g., weekly screening log, routine electronic data submissions).\nUnder specific direction, assist w/ data entry of study activity onto a case report form (paper or electronic), & maintain a database program to track all study activity (i.e., study enrollment & consents). Assist in maintaining research charts & site regulatory files. Assist in ensuring that study-related, non-test article supplies are shipped & re-supplied according to protocol, including tracking expiration dates. Assist in obtaining protocol specific regulatory documents including MDs/RNs licenses & CVs.\nReport any potential protocol violations/deviations to the PI in a timely manner. Support the regulatory team in the maintenance & storage of critical documents required to be maintained & provided to the Sponsor during the conduct of the trial. W/ supervision, assist in study closure activities including preparation & collection of close-out documentation, & preparation of study files for distribution to longterm storage.\nThis job description is not all encompassing.\nBasic Qualifications:\nExperience\nN/A.\nEducation\nAssociate's degree or higher OR two (2) years of work experience in an ambulatory and/or acute health care setting required (LVN maybe substituted for an Associate's degree)High School Diploma or General Education Development (GED) required.\nLicense, Certification, Registration\nN/A.\nAdditional Requirements:\nWillingness to obtain IATA/DOT certification prior to start date.Current BLS certification required prior to start date.Satisfies requirements for career advancement/maintenance as defined by the Clinical Trial Career Ladder Program.Must be proficient in electronic health systems and databases used in research environment and word-processing or willingness to learn within 3 months of hire.Demonstrate organizational and communication skills.Demonstrate written, verbal, and interpersonal communication skills.Demonstrate proficiency in medical terminology.Demonstrate attention to detail and accuracy.Ability to manage multiple tasks.Demonstrate good prioritization and organizational skills.Ability to be flexible and dependable.Ability to work effectively on cross-functional teams.Present professional manner and appearance.Demonstrated skill in administrative tasks (i.e., filing, photocopying, faxing, etc.).Knowledge of GCP, federal, state, and local regulations including HIPAA and KP policies and procedures.Must be able to work in a Labor Partnership environment.\nPreferred Qualifications:\nClinical trials experience preferred.\nPrimaryLocation : California,South San Francisco,S. San Francisco Medical Offices\nHoursPerWeek : 40\nShift : Day\nWorkdays : Variable\nWorkingHoursStart : 08:30 AM\nWorkingHoursEnd : 05:00 PM\nJob Schedule : Full-time\nJob Type : Standard\nEmployee Status : Regular\nEmployee Group/Union Affiliation : CNV-NUE|NUE|Non Union Employee\nJob Level : Entry Level\nJob Category : Research and Development\nDepartment : Oakland Reg - 2000 Broadway - Rsrch-Invstgtr Staffg Mdl Dept - 0201\nTravel : No\nKaiser Permanente is an equal opportunity employer committed to a diverse and inclusive workforce. Applicants will receive consideration for employment without regard to race, color, religion, sex (including pregnancy), age, sexual orientation, national origin, marital status, parental status, ancestry, disability, gender identity, veteran status, genetic information, other distinguishing characteristics of diversity and inclusion, or any other protected status.\n\nExternal hires must pass a background check/drug screen. Qualified applicants with arrest and/or conviction records will be considered for employment in a manner consistent with federal and state laws, as well as applicable local ordinances, including but not limited to the San Francisco and Los Angeles Fair Chance Ordinances
'''

In [37]:
# A list of text
text = [resume, job_description]

In [38]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(text)

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

#Print the similarity scores
print("\nSimilarity Scores:")
print(cosine_similarity(count_matrix))


Similarity Scores:
[[1.         0.58425224]
 [0.58425224 1.        ]]


In [40]:
#get the match percentage
matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
matchPercentage = round(matchPercentage, 2) # round to two decimal
print("Your resume matches about "+ str(matchPercentage)+ "% of the job description.")

Your resume matches about 58.43% of the job description.


In [41]:
def jobmatch(description):
    text = [resume, description]
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(text)
    matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
    matchPercentage = round(matchPercentage, 2)
    return matchPercentage

In [48]:
df.dropna(inplace=True)

In [49]:
df['match_rate'] = df['description'].apply(lambda x: jobmatch(x))

In [54]:
df.drop(columns=['description', 'summary'])

,title,company,location,salary,rating,desc_summarizer,match_rate
0,"Clinical Trial Research Coordinator I - South San Francisco,..._new",Kaiser Permanente,"San Francisco, CA",None,4.1,"[{'summary_text': ' The Clinical Trial Research Coordinator I (CTRC-I) assists in the operational and administrative research support of the Kaiser Permanente Northern California (KPNC) PI and KPNC Clinical Trials Operations Director or designee . This position is also responsible for assisting in protecting the health, safety, and welfare of research participants .'}]",58.43
1,Senior Research Assistant_new,Kaiser Permanente,"Oakland, CA",None,4.1,"[{'summary_text': "" Assists in training new study staff in protocols and implementation of study goals/objectives . Conducts telephone or in-person interviews with study participants and providers . Assists with all aspects of mailed surveys . Monitors participants' progress (e.g., blood pressure, exerciaries)""}]",48.32
2,Data Analyst_new,National Mortgage Insurance Corporation,"Emeryville, CA 94608",None,None,"[{'summary_text': ' The Analyst will have broad organizational access to support the analytic and reporting needs at all levels of Sales and Risk Management . He will be part of the Sales Effectiveness team with responsibility for owning and developing reporting and analysis tools . The Analyst is responsible for developing and maintaining large analytic datasets, both internal and externally sourced .'}]",56.51
3,"SR Product Analyst - Chartio, SQL (Part-time Contract/REMOTE..._new",CyberCoders,"San Francisco, CA 94107 (South Of Market area)",None,3.2,[{'summary_text': ' We are looking for a contractor & expert product analyst/developer to refresh/rebuild our dashboards and assist with the backlog of changes requested by our internal teams . We built the dashboards in-house but are running into errors and limitations due to the climbing amount of users and data . This person will not be tasked with analyzing data .'}],54.32
4,Sr. PHP Developer_new,The Lead Group,United States,None,None,"[{'summary_text': ' We are looking to bring on a Sr. PHP Developer to help us build new features that empower our business efforts . We are a rapidly growing Online Lead Generation company with offices in Kansas City & Atlanta, and remote staff in Dallas & Phoenix .Responsibilities include writing “clean”, well-designed code that meets all provided product specifications .'}]",60.84
...,...,...,...,...,...,...,...
807,Shelter Advocate/Data Entry Specialist,Confidential Nonprofit,"Oakland, CA",$17 - $18 an hour,None,"[{'summary_text': ' Applicants must have:Relative experience working with diverse populations in residential setting . At least 2-3 years of social service and/or residential program experience . Clean DMV record, proof of auto insurance allowing use of personal car for business/work and satisfactory criminal background check is a must .'}]",34.24
808,Head Statistics & Data Management,Quantum Leap Healthcare,"San Francisco Bay Area, CA",None,None,"[{'summary_text': ' The manager of statistics and data management if responsible for managing the functions of biostatistics, statistical programming, and data . These clinical trials may be Phase I, II and III pharmaceutical trials and medical device trials evaluating new biomarkers for cancer . Candidates must have previous personnel, function and project management experience .'}]",59.45
809,Program Manager (AEA West),"Association For Energy Affordability, Inc.","Emeryville, CA",None,None,"[{'summary_text': ' Association for Energy Affordability (AEA) is a rapidly growing nonprofit energy services and training organization dedicated to achieving energy efficiency in buildings . The candidate must be passionate about energy efficiency and green building, highly organized, and have strength in data analytics . The Program Manager will oversee the day-today management and process flow of programs directed at providing energy efficiency services to multifa